# Import Libraries

In [1]:
import numpy as np
import scipy
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import cross_val_score, GridSearchCV
np.random.seed(0)

# Signal Preprocessing

In [2]:
#Bandpass Filter 
def FilterBandPass(CNT, Wn, Fs):    
    Wn = np.array([Wn[0] / (Fs/2), Wn[1] / (Fs/2)])
    
    # Filter Design
    order = 3
    type = 'bandpass'
    b, a = scipy.signal.butter(order, Wn, btype = 'bandpass')
    
    # Apply Filter
    CNT = scipy.signal.filtfilt(b, a, CNT, axis = 0)
    
    return CNT

In [3]:
# CAR Filter for Spatial Filtering
def CARFilter(Data):
    '''
        Implementation of Common Average Filter for Temporal Resolution Improvement 
        
        Inputs:
            Data ==> EEG Data of a Single Trial in Shape: (Signal, Channels)
            
        Output:
            Data ==> Filtered Data
    '''
    M = np.mean(Data, axis = 1)
    
    for i in range(Data.shape[1]):
        Data[:, i] = Data[:, i] - M
    
    return Data

# Design CSP Algorithm

In [4]:
def CSP(Train1, Train2, m):
    '''
        Implementation of Common Spatial Pattern Algorithm for Motor Imagery BCI

        Inputs:
            Train1 ==> First Class Train Data in Shape: (Signal, Channels, Trials)
            Train2 ==> Second Class Train Data in Shape: (Signal, Channels, Trials)
            m      ==> Number of Channels
            
        Outputs:
            W      ==> CSP Transform Matrix in Shape: (Channels, Channels)
    '''
    
    # First Class Covariance Matrix Calculation
    Rh = 0
    for i in range(Train1.shape[2]):
        Data = Train1[:, :, i]

        # Normalize Data
        M = np.mean(Data, axis = 0)

        for i in range(Data.shape[0]):
            Data[i, :] = Data[i, :] - M

        #  Calculation Cov Matrix and Sum all
        Rh += Data.T @ Data / np.trace(Data.T @ Data)

    # First Class Covariance Matrix Calculation
    Rf = 0
    for i in range(Train2.shape[2]):
        Data = Train2[:, :, i]

        # Normalize Data
        M = np.mean(Data, axis = 0)

        for i in range(Data.shape[0]):
            Data[i, :] = Data[i, :] - M

        #  Calculation Cov Matrix and Sum all
        Rf += Data.T @ Data / np.trace(Data.T @ Data)


    # Normalize Cov Matrix
    Rf /= Train2.shape[2]
    Rh /= Train1.shape[2]

    # Eigen Value Decomposition
    u, v = scipy.linalg.eig(Rh, Rf)
    # u = np.real(u)

    # Sort eigenvalues and eigenvectors by eigenvalues
    sorted_indices = np.argsort(u)[::-1]  # Sort in descending order
    u = u[sorted_indices]
    v = v[:, sorted_indices]

    Wm = list(range(m)) + list(range(v.shape[1] - m, v.shape[1]))
    W = v[:, Wm]

    return W

# Multi Class CSP Function Using One-Vs-All Method

In [5]:
def MultiClassCSPOVA(X1Train, X2Train, X3Train, X4Train, M):
    '''
        
        
        Inputs:
            X1Train ==> Train Data for First Class in Shape: (Signal, Channels, Trials)
            X2Train ==> Train Data for Second Class in Shape: (Signal, Channels, Trials)
            X3Train ==> Train Data for Third Class in Shape: (Signal, Channels, Trials)
            X4Train ==> Train Data for Fourth Class in Shape: (Signal, Channels, Trials)
            
    '''
    
    Data1 = X1Train
    Data2 = np.concatenate((X2Train, X3Train, X4Train), axis = 2)
    W1 = CSP(Data1, Data2, M)
    
    Data1 = X2Train
    Data2 = np.concatenate((X1Train, X3Train, X4Train), axis = 2)
    W2 = CSP(Data1, Data2, M)
    
    Data1 = X3Train
    Data2 = np.concatenate((X1Train, X2Train, X4Train), axis = 2)
    W3 = CSP(Data1, Data2, M)
    
    Data1 = X4Train
    Data2 = np.concatenate((X2Train, X3Train, X1Train), axis = 2)
    W4 = CSP(Data1, Data2, M)
    
    W = list([W1, W2, W3, W4])
    
    return W

# Multi Class CSP Function Using One-Vs-One Method

In [6]:
def MultiClassCSPOVO(X1Train, X2Train, X3Train, X4Train, M):
    '''
        
        Inputs:
            X1Train ==> Train Data for First Class in Shape: (Signal, Channels, Trials)
            X2Train ==> Train Data for Second Class in Shape: (Signal, Channels, Trials)
            X3Train ==> Train Data for Third Class in Shape: (Signal, Channels, Trials)
            X4Train ==> Train Data for Fourth Class in Shape: (Signal, Channels, Trials)
            
    '''
    
    W12 = CSP(X1Train, X2Train, M)
    
    W13 = CSP(X1Train, X3Train, M)
    
    W14 = CSP(X1Train, X4Train, M)
    
    W23 = CSP(X2Train, X3Train, M) 
    
    W24 = CSP(X2Train, X4Train, M)
    
    W34 = CSP(X3Train, X4Train, M)
    
    W = list([W12, W13, W14, W23, W24, W34])
    
    return W

# Multi Class CSP in One-Vs-One Method

In [7]:
AvailableData = '12456789'
# AvailableData = '1'
AccAll = []

for c in AvailableData:
    # Create File Name
    FileName = 'A' + c + '.mat'
    
    # Read Data from File
    Data = scipy.io.loadmat(FileName)
    X1 = Data['X1']
    X2 = Data['X2']
    X3 = Data['X3']
    X4 = Data['X4']
    Fs = Data['Fs'][0][0]
    
    
    ChannelNum = X1.shape[1]
    ChannelLen = X1.shape[0]
    TrialNum   = X1.shape[2]
    
    # Apply Bandpass Filter
    wn = [8, 25]
    for i in range(TrialNum):
        X1[:, :, i] = FilterBandPass(X1[:, :, i], wn, Fs)
        X2[:, :, i] = FilterBandPass(X2[:, :, i], wn, Fs)
        X3[:, :, i] = FilterBandPass(X3[:, :, i], wn, Fs)
        X4[:, :, i] = FilterBandPass(X4[:, :, i], wn, Fs)
        
    #Apply CAR Filter
    for j in range(TrialNum):
        X1[:, :, i] = CARFilter(X1[:, :, i])
        X2[:, :, i] = CARFilter(X2[:, :, i])
        X3[:, :, i] = CARFilter(X3[:, :, i])
        X4[:, :, i] = CARFilter(X4[:, :, i])
        
    
    # Separate Data into Test and Train
    Div = 0.8
    IndTrain = list(range(round(Div * X1.shape[2])))
    IndTest = list(range(IndTrain[-1] + 1, TrialNum))
    
    X1Test = X1[:, :, IndTest]
    X1Train = X1[:, :, IndTrain]
    
    X2Test = X2[:, :, IndTest]
    X2Train = X2[:, :, IndTrain]
    
    X3Test = X3[:, :, IndTest]
    X3Train = X3[:, :, IndTrain]
    
    X4Test = X4[:, :, IndTest]
    X4Train = X4[:, :, IndTrain]
    
    # Apply MultiClass CSP  
    M = 4
    W = MultiClassCSPOVA(X1Train, X2Train, X3Train, X4Train, M)
    
    ## Freature Extract
    # Train Feature Extraction
    FTrainX1 = []
    FTrainX2 = []
    FTrainX3 = []
    FTrainX4 = []
    
    # X1 Freature Extract
    for i in range(X1Train.shape[2]):
        tmpData = X1Train[:, :, i]
        
        Ftmp = []
        for j in range(len(W)):
            w = W[j]
            tmp = w.T @ tmpData.T
            Ftmp.append(np.log10(np.var(tmp, axis = 1) / np.sum(np.var(tmp, axis = 1))))
            
        Ftmp = np.array(Ftmp).reshape(-1)
        FTrainX1.append(Ftmp)
    
    # X2 Freature Extract
    for i in range(X2Train.shape[2]):
        tmpData = X2Train[:, :, i]
        
        Ftmp = []
        for j in range(len(W)):
            w = W[j]
            tmp = w.T @ tmpData.T
            Ftmp.append(np.log10(np.var(tmp, axis = 1) / np.sum(np.var(tmp, axis = 1))))
            
        Ftmp = np.array(Ftmp).reshape(-1)
        FTrainX2.append(Ftmp)
        
    # X3 Freature Extract
    for i in range(X3Train.shape[2]):
        tmpData = X3Train[:, :, i]
        
        Ftmp = []
        for j in range(len(W)):
            w = W[j]
            tmp = w.T @ tmpData.T
            Ftmp.append(np.log10(np.var(tmp, axis = 1) / np.sum(np.var(tmp, axis = 1))))
            
        Ftmp = np.array(Ftmp).reshape(-1)
        FTrainX3.append(Ftmp)
    
    # X4 Freature Extract
    for i in range(X4Train.shape[2]):
        tmpData = X4Train[:, :, i]
        
        Ftmp = []
        for j in range(len(W)):
            w = W[j]
            tmp = w.T @ tmpData.T
            Ftmp.append(np.log10(np.var(tmp, axis = 1) / np.sum(np.var(tmp, axis = 1))))
            
        Ftmp = np.array(Ftmp).reshape(-1)
        FTrainX4.append(Ftmp)
        
    
    FTrainX1 = np.array(FTrainX1)    
    FTrainX2 = np.array(FTrainX2) 
    FTrainX3 = np.array(FTrainX3) 
    FTrainX4 = np.array(FTrainX4) 

    
    # Test Feature Extraction
    FTestX1 = []
    FTestX2 = []
    FTestX3 = []
    FTestX4 = []
    
    # X1 Freature Extract
    for i in range(X1Test.shape[2]):
        tmpData = X1Test[:, :, i]
        
        Ftmp = []
        for j in range(len(W)):
            w = W[j]
            tmp = w.T @ tmpData.T
            Ftmp.append(np.log10(np.var(tmp, axis = 1) / np.sum(np.var(tmp, axis = 1))))
            
        Ftmp = np.array(Ftmp).reshape(-1)
        FTestX1.append(Ftmp)
    
    # X2 Freature Extract
    for i in range(X2Test.shape[2]):
        tmpData = X2Test[:, :, i]
        
        Ftmp = []
        for j in range(len(W)):
            w = W[j]
            tmp = w.T @ tmpData.T
            Ftmp.append(np.log10(np.var(tmp, axis = 1) / np.sum(np.var(tmp, axis = 1))))
            
        Ftmp = np.array(Ftmp).reshape(-1)
        FTestX2.append(Ftmp)
        
    # X3 Freature Extract
    for i in range(X3Test.shape[2]):
        tmpData = X3Test[:, :, i]
        
        Ftmp = []
        for j in range(len(W)):
            w = W[j]
            tmp = w.T @ tmpData.T
            Ftmp.append(np.log10(np.var(tmp, axis = 1) / np.sum(np.var(tmp, axis = 1))))
            
        Ftmp = np.array(Ftmp).reshape(-1)
        FTestX3.append(Ftmp)
    
    # X4 Freature Extract
    for i in range(X4Test.shape[2]):
        tmpData = X4Test[:, :, i]
        
        Ftmp = []
        for j in range(len(W)):
            w = W[j]
            tmp = w.T @ tmpData.T
            Ftmp.append(np.log10(np.var(tmp, axis = 1) / np.sum(np.var(tmp, axis = 1))))
            
        Ftmp = np.array(Ftmp).reshape(-1)
        FTestX4.append(Ftmp)
        
    
    FTestX1 = np.array(FTestX1)    
    FTestX2 = np.array(FTestX2) 
    FTestX3 = np.array(FTestX3) 
    FTestX4 = np.array(FTestX4)
    
    # Final Train and Test Data Creation
    TrainData  = np.concatenate((FTrainX1, FTrainX2, FTrainX3, FTrainX4), axis = 0)
    TestData   = np.concatenate((FTestX1, FTestX2, FTestX3, FTestX4), axis = 0)
    TrainLabel = np.concatenate((np.ones(FTrainX1.shape[0]), 2 * np.ones(FTrainX1.shape[0]), 3 * np.ones(FTrainX1.shape[0]), 4 * np.ones(FTrainX1.shape[0])))
    TestLabel  = np.concatenate((np.ones(FTestX1.shape[0]), 2 * np.ones(FTestX1.shape[0]), 3 * np.ones(FTestX1.shape[0]), 4 * np.ones(FTestX1.shape[0])))
        
    # Train Model
    
    # params_grid = {
    #     'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],  
    #     'gamma': [1, 0.1, 0.01, 0.001, 0.0001, 'scale', 'auto'], 
    #     'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    #     'degree': [0, 1, 2, 3, 4, 5, 6], # only used when kernel is 'poly'
    #     'coef0': [0.0, 0.1, 0.5, 1.0] # only used in 'poly' and 'sigmoid'
    # }

    # svm_model = GridSearchCV(svm.SVC(), params_grid, n_jobs = -1)
    # svm_model.fit(TrainData, TrainLabel)

    # mdl = svm_model.best_estimator_
    
    mdl = svm.SVC(kernel='linear', C = 1.0)
    mdl.fit(TrainData, TrainLabel)
    
    # Test Model
    LabelPredict = mdl.predict(TestData)

    Acc = accuracy_score(TestLabel, LabelPredict)
    AccAll.append(Acc)
    ConfMat = confusion_matrix(TestLabel, LabelPredict)
    
    # Print Metrics
    print(f'Subject {c}')
    print("Accuracy : ", accuracy_score(TestLabel, LabelPredict))
    print(f"Class 1 Accuracy : {(np.diag(ConfMat) / ConfMat.sum(axis=1))[0] * 100}")
    print(f"Class 2 Accuracy : {(np.diag(ConfMat) / ConfMat.sum(axis=1))[1] * 100}")
    print(f"Class 2 Accuracy : {(np.diag(ConfMat) / ConfMat.sum(axis=1))[2] * 100}")
    print(f"Class 2 Accuracy : {(np.diag(ConfMat) / ConfMat.sum(axis=1))[3] * 100}")
    print("Confusion Matrix : \n", ConfMat)
    print('-----------------------------------------------------------------')   
print("AccAll : ", np.mean(AccAll))

Subject 1
Accuracy :  0.5178571428571429
Class 1 Accuracy : 64.28571428571429
Class 2 Accuracy : 50.0
Class 2 Accuracy : 21.428571428571427
Class 2 Accuracy : 71.42857142857143
Confusion Matrix : 
 [[ 9  5  0  0]
 [ 7  7  0  0]
 [ 3  2  3  6]
 [ 1  2  1 10]]
-----------------------------------------------------------------
Subject 2
Accuracy :  0.32142857142857145
Class 1 Accuracy : 21.428571428571427
Class 2 Accuracy : 57.14285714285714
Class 2 Accuracy : 28.57142857142857
Class 2 Accuracy : 21.428571428571427
Confusion Matrix : 
 [[3 4 4 3]
 [1 8 1 4]
 [3 5 4 2]
 [6 4 1 3]]
-----------------------------------------------------------------
Subject 4
Accuracy :  0.5178571428571429
Class 1 Accuracy : 35.714285714285715
Class 2 Accuracy : 85.71428571428571
Class 2 Accuracy : 57.14285714285714
Class 2 Accuracy : 28.57142857142857
Confusion Matrix : 
 [[ 5  7  2  0]
 [ 2 12  0  0]
 [ 0  4  8  2]
 [ 2  2  6  4]]
-----------------------------------------------------------------
Subject 5
Acc

## Multi Class CSP in One-Vs-One Method 

In [8]:
AvailableData = '12456789'
# AvailableData = '1'
AccAllO = []

for c in AvailableData:
    # Create File Name
    FileName = 'A' + c + '.mat'
    
    # Read Data from File
    Data = scipy.io.loadmat(FileName)
    X1 = Data['X1']
    X2 = Data['X2']
    X3 = Data['X3']
    X4 = Data['X4']
    Fs = Data['Fs'][0][0]
    
    
    ChannelNum = X1.shape[1]
    ChannelLen = X1.shape[0]
    TrialNum   = X1.shape[2]
    
    # Apply Bandpass Filter
    wn = [8, 25]
    for i in range(TrialNum):
        X1[:, :, i] = FilterBandPass(X1[:, :, i], wn, Fs)
        X2[:, :, i] = FilterBandPass(X2[:, :, i], wn, Fs)
        X3[:, :, i] = FilterBandPass(X3[:, :, i], wn, Fs)
        X4[:, :, i] = FilterBandPass(X4[:, :, i], wn, Fs)
        
        
    # Apply CAR Filter
    for i in range(TrialNum):
        X1[:, :, i] = CARFilter(X1[:, :, i])
        X2[:, :, i] = CARFilter(X2[:, :, i])
        X3[:, :, i] = CARFilter(X3[:, :, i])
        X4[:, :, i] = CARFilter(X4[:, :, i])
    
    # Separate Data into Test and Train
    Div = 0.8
    IndTrain = list(range(round(Div * X1.shape[2])))
    IndTest = list(range(IndTrain[-1] + 1, TrialNum))
    
    X1Test = X1[:, :, IndTest]
    X1Train = X1[:, :, IndTrain]
    
    X2Test = X2[:, :, IndTest]
    X2Train = X2[:, :, IndTrain]
    
    X3Test = X3[:, :, IndTest]
    X3Train = X3[:, :, IndTrain]
    
    X4Test = X4[:, :, IndTest]
    X4Train = X4[:, :, IndTrain]
    
    # Apply MultiClass CSP  
    M = 4
    W = MultiClassCSPOVO(X1Train, X2Train, X3Train, X4Train, M)
    
    ## Freature Extract
    # Train Feature Extraction
    FTrainX1 = []
    FTrainX2 = []
    FTrainX3 = []
    FTrainX4 = []
    
    # X1 Freature Extract
    for i in range(X1Train.shape[2]):
        tmpData = X1Train[:, :, i]
        
        Ftmp = []
        for j in range(len(W)):
            w = W[j]
            tmp = w.T @ tmpData.T
            Ftmp.append(np.log10(np.var(tmp, axis = 1) / np.sum(np.var(tmp, axis = 1))))
            
        Ftmp = np.array(Ftmp).reshape(-1)
        FTrainX1.append(Ftmp)
    
    # X2 Freature Extract
    for i in range(X2Train.shape[2]):
        tmpData = X2Train[:, :, i]
        
        Ftmp = []
        for j in range(len(W)):
            w = W[j]
            tmp = w.T @ tmpData.T
            Ftmp.append(np.log10(np.var(tmp, axis = 1) / np.sum(np.var(tmp, axis = 1))))
            
        Ftmp = np.array(Ftmp).reshape(-1)
        FTrainX2.append(Ftmp)
        
    # X3 Freature Extract
    for i in range(X3Train.shape[2]):
        tmpData = X3Train[:, :, i]
        
        Ftmp = []
        for j in range(len(W)):
            w = W[j]
            tmp = w.T @ tmpData.T
            Ftmp.append(np.log10(np.var(tmp, axis = 1) / np.sum(np.var(tmp, axis = 1))))
            
        Ftmp = np.array(Ftmp).reshape(-1)
        FTrainX3.append(Ftmp)
    
    # X4 Freature Extract
    for i in range(X4Train.shape[2]):
        tmpData = X4Train[:, :, i]
        
        Ftmp = []
        for j in range(len(W)):
            w = W[j]
            tmp = w.T @ tmpData.T
            Ftmp.append(np.log10(np.var(tmp, axis = 1) / np.sum(np.var(tmp, axis = 1))))
            
        Ftmp = np.array(Ftmp).reshape(-1)
        FTrainX4.append(Ftmp)
        
    
    FTrainX1 = np.array(FTrainX1)    
    FTrainX2 = np.array(FTrainX2) 
    FTrainX3 = np.array(FTrainX3) 
    FTrainX4 = np.array(FTrainX4) 

    
    # Test Feature Extraction
    FTestX1 = []
    FTestX2 = []
    FTestX3 = []
    FTestX4 = []
    
    # X1 Freature Extract
    for i in range(X1Test.shape[2]):
        tmpData = X1Test[:, :, i]
        
        Ftmp = []
        for j in range(len(W)):
            w = W[j]
            tmp = w.T @ tmpData.T
            Ftmp.append(np.log10(np.var(tmp, axis = 1) / np.sum(np.var(tmp, axis = 1))))
            
        Ftmp = np.array(Ftmp).reshape(-1)
        FTestX1.append(Ftmp)
    
    # X2 Freature Extract
    for i in range(X2Test.shape[2]):
        tmpData = X2Test[:, :, i]
        
        Ftmp = []
        for j in range(len(W)):
            w = W[j]
            tmp = w.T @ tmpData.T
            Ftmp.append(np.log10(np.var(tmp, axis = 1) / np.sum(np.var(tmp, axis = 1))))
            
        Ftmp = np.array(Ftmp).reshape(-1)
        FTestX2.append(Ftmp)
        
    # X3 Freature Extract
    for i in range(X3Test.shape[2]):
        tmpData = X3Test[:, :, i]
        
        Ftmp = []
        for j in range(len(W)):
            w = W[j]
            tmp = w.T @ tmpData.T
            Ftmp.append(np.log10(np.var(tmp, axis = 1) / np.sum(np.var(tmp, axis = 1))))
            
        Ftmp = np.array(Ftmp).reshape(-1)
        FTestX3.append(Ftmp)
    
    # X4 Freature Extract
    for i in range(X4Test.shape[2]):
        tmpData = X4Test[:, :, i]
        
        Ftmp = []
        for j in range(len(W)):
            w = W[j]
            tmp = w.T @ tmpData.T
            Ftmp.append(np.log10(np.var(tmp, axis = 1) / np.sum(np.var(tmp, axis = 1))))
            
        Ftmp = np.array(Ftmp).reshape(-1)
        FTestX4.append(Ftmp)
        
    
    FTestX1 = np.array(FTestX1)    
    FTestX2 = np.array(FTestX2) 
    FTestX3 = np.array(FTestX3) 
    FTestX4 = np.array(FTestX4)
    
    # Final Train and Test Data Creation
    TrainData  = np.concatenate((FTrainX1, FTrainX2, FTrainX3, FTrainX4), axis = 0)
    TestData   = np.concatenate((FTestX1, FTestX2, FTestX3, FTestX4), axis = 0)
    TrainLabel = np.concatenate((np.ones(FTrainX1.shape[0]), 2 * np.ones(FTrainX1.shape[0]), 3 * np.ones(FTrainX1.shape[0]), 4 * np.ones(FTrainX1.shape[0])))
    TestLabel  = np.concatenate((np.ones(FTestX1.shape[0]), 2 * np.ones(FTestX1.shape[0]), 3 * np.ones(FTestX1.shape[0]), 4 * np.ones(FTestX1.shape[0])))
        
    # Train Model
    
    # params_grid = {
    #     'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],  
    #     'gamma': [1, 0.1, 0.01, 0.001, 0.0001, 'scale', 'auto'], 
    #     'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    #     'degree': [0, 1, 2, 3, 4, 5, 6], # only used when kernel is 'poly'
    #     'coef0': [0.0, 0.1, 0.5, 1.0] # only used in 'poly' and 'sigmoid'
    # }

    # svm_model = GridSearchCV(svm.SVC(), params_grid, n_jobs = -1)
    # svm_model.fit(TrainData, TrainLabel)

    # mdl = svm_model.best_estimator_
    
    mdl = svm.SVC(kernel='rbf', C = 1.0)
    mdl.fit(TrainData, TrainLabel)
    
    # Test Model
    LabelPredict = mdl.predict(TestData)

    Acc = accuracy_score(TestLabel, LabelPredict)
    ConfMat = confusion_matrix(TestLabel, LabelPredict)
    AccAllO.append(Acc)
    
    # Print Metrics
    print(f'Subject {c}')
    print("Accuracy : ", accuracy_score(TestLabel, LabelPredict))
    print(f"Class 1 Accuracy : {(np.diag(ConfMat) / ConfMat.sum(axis=1))[0] * 100}")
    print(f"Class 2 Accuracy : {(np.diag(ConfMat) / ConfMat.sum(axis=1))[1] * 100}")
    print(f"Class 3 Accuracy : {(np.diag(ConfMat) / ConfMat.sum(axis=1))[2] * 100}")
    print(f"Class 4 Accuracy : {(np.diag(ConfMat) / ConfMat.sum(axis=1))[3] * 100}")
    print("Confusion Matrix : \n", ConfMat)
    print('-----------------------------------------------------------------')  
print("Mean Acc of All Subjects: ", np.mean(AccAllO))

Subject 1
Accuracy :  0.4642857142857143
Class 1 Accuracy : 100.0
Class 2 Accuracy : 0.0
Class 3 Accuracy : 14.285714285714285
Class 4 Accuracy : 71.42857142857143
Confusion Matrix : 
 [[14  0  0  0]
 [14  0  0  0]
 [ 6  0  2  6]
 [ 3  0  1 10]]
-----------------------------------------------------------------
Subject 2
Accuracy :  0.3392857142857143
Class 1 Accuracy : 85.71428571428571
Class 2 Accuracy : 21.428571428571427
Class 3 Accuracy : 21.428571428571427
Class 4 Accuracy : 7.142857142857142
Confusion Matrix : 
 [[12  0  1  1]
 [ 8  3  3  0]
 [ 9  2  3  0]
 [11  0  2  1]]
-----------------------------------------------------------------
Subject 4
Accuracy :  0.375
Class 1 Accuracy : 21.428571428571427
Class 2 Accuracy : 64.28571428571429
Class 3 Accuracy : 42.857142857142854
Class 4 Accuracy : 21.428571428571427
Confusion Matrix : 
 [[3 6 5 0]
 [3 9 1 1]
 [0 6 6 2]
 [1 5 5 3]]
-----------------------------------------------------------------
Subject 5
Accuracy :  0.28571428571428